In [ ]:
notebook_id = ''
notebook_source = 'GIT_REPO'
notebook_uri = ''
git_branch = ''
git_commit = ''
target_file = ''
package_management_file = ''
docker_registry = ''
image_version = ''

In [ ]:
assert notebook_id, 'notebook_id not defined'
assert notebook_uri, 'git_repo not defined'
assert target_file, 'target_file not defined'
assert docker_registry, 'docker_registry not defined'
assert image_version, 'image_version not defined'

if notebook_source == 'GIT_REPO':
    assert git_branch, 'branch not defined'


In [ ]:
docker_repo = f'{docker_registry}/training'
image_tag = f'{notebook_id}_{image_version}'
notebook_dir = 'notebook_dir'

In [ ]:
assert docker_repo, 'docker_repo not defined'
assert image_tag, 'image_tag not defined'

In [ ]:
download_notebook_cmd = f'&& git clone --single-branch -b master {notebook_uri} {notebook_dir} \\'
if notebook_source == 'S3_FILES':
    download_notebook_cmd = f'aws s3 cp --recursive {notebook_uri} {notebook_dir}'

install_packages_cmd = ''
if package_management_file:
    install_packages_cmd = f'''
    && cat {package_management_file} \\
    && while read requirement; do conda run -n ${{runtime}} conda install -y ${{requirement}} || conda run -n ${{runtime}} pip install --user ${{requirement}}; done < {package_management_file}
'''

dockerfile = f'''
FROM gn00023040/notebook-runtime:1.0

RUN set -xe \\
    {download_notebook_cmd}
    && cd {notebook_dir} \\
    && lang=$(cat {target_file} | jq -r '.metadata.language_info.name') \\
    && lang_ver=$(cat {target_file} | jq -r '.metadata.language_info.version') \\
    && runtime=${{lang}}_${{lang_ver}} \\
    && kernel=$(cat {target_file} | jq -r '.metadata.kernelspec.name') \\
    && conda create -y -n ${{runtime}} ${{lang}}=${{lang_ver}} \\
    && conda run -n ${{runtime}} python --version \\
    && conda run -n ${{runtime}} conda install -y ipykernel \\
    && conda run -n ${{runtime}} python -m ipykernel install --user --name=${{kernel}} \\
    {install_packages_cmd}
    && echo 'done'
'''

with open('Dockerfile', 'w') as file:
    file.write(dockerfile)

In [ ]:
!set -xe \
    && cat Dockerfile \
    && sudo docker build -t {docker_repo}:{image_tag} . \
    && aws ecr get-login-password --region us-east-2 | sudo docker login --username AWS --password-stdin {docker_registry} \
    && sudo docker push {docker_repo}:{image_tag}